<a href="https://colab.research.google.com/github/vkt1414/Cloud-Resources-Workflows/blob/main/Notebooks/Totalsegmentator/slicerMappingsTotalSegmentator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Importing Packages**

In [9]:
import pandas as pd
import json
import yaml
import os
import sys
from pathlib import Path
import time
import ast

###**Runtime Environment**

In [10]:
curr_dir   = Path().absolute()
os.environ['TZ'] = 'US/Eastern'
time.tzset()
current_time = time.strftime('%a %b %d %H:%M:%S %Y', time.localtime())
print(current_time)
print("\nCurrent directory :{}".format( curr_dir))
print("Python version    :", sys.version.split('\n')[0])

Fri Apr 07 11:16:58 2023

Current directory :/content
Python version    : 3.9.16 (main, Dec  7 2022, 01:11:51) 


###**Download TotalSegmentator mappings from Slicer Documentation**

In [11]:
try:
  os.remove(f'{curr_dir}/TotalSegmentator.py')
except OSError:
  pass
!wget https://raw.githubusercontent.com/lassoan/SlicerTotalSegmentator/main/TotalSegmentator/TotalSegmentator.py

--2023-04-07 11:17:01--  https://raw.githubusercontent.com/lassoan/SlicerTotalSegmentator/main/TotalSegmentator/TotalSegmentator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66062 (65K) [text/plain]
Saving to: ‘TotalSegmentator.py’

TotalSegmentator.py 100%[===================>]  64.51K  --.-KB/s    in 0.006s  

2023-04-07 11:17:02 (10.7 MB/s) - ‘TotalSegmentator.py’ saved [66062/66062]



###**Data Wrangling**

In [12]:
with open('TotalSegmentator.py', 'r') as f:
    content = f.read()
    tree = ast.parse(content)

class TotalSegmentatorLogicFinder(ast.NodeVisitor):
    def __init__(self):
        self.result = None

    def visit_ClassDef(self, node):
        if node.name == 'TotalSegmentatorLogic':
            for body_node in node.body:
                if isinstance(body_node, ast.FunctionDef) and body_node.name == '__init__':
                    for expr in body_node.body:
                        if isinstance(expr, ast.Assign):
                            target = expr.targets[0]
                            if isinstance(target, ast.Attribute) and target.attr == 'totalSegmentatorLabelTerminology':
                                self.result = ast.literal_eval(ast.unparse(expr.value))
        self.generic_visit(node)

finder = TotalSegmentatorLogicFinder()
finder.visit(tree)
totalSegmentatorLabelTerminology = finder.result
data = []
for key, value in totalSegmentatorLabelTerminology.items():
    fields = value.split('~')
    row = {'Structure': key}
    for i, field in enumerate(fields):
        subfields = field.split('^')
        if i == 0:
            if len(subfields) > 1:
                row['SegmentAlgorithmName'] = subfields[0]
                row['SegmentAlgorithmType'] = subfields[1]
        elif i == 1:
            if len(subfields) > 2:
                row['SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyCategoryCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyCategoryCodeSequence.CodeMeaning'] = subfields[2]
        elif i == 2:
            if len(subfields) > 2:
                row['SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyTypeCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyTypeCodeSequence.CodeMeaning'] = subfields[2]
        elif i == 3:
            if len(subfields) > 2:
                row['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'] = subfields[0]
                row['SegmentedPropertyTypeModifierCodeSequence.CodeValue'] = subfields[1]
                row['SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'] = subfields[2]
    data.append(row)

df = pd.DataFrame(data)
df=df.sort_values('Structure').reset_index(drop=True)
df

,Structure,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning
0,adrenal_gland_left,SCT,123037004,Anatomical Structure,SCT,23451007,Adrenal gland,SCT,7771000,Left
1,adrenal_gland_right,SCT,123037004,Anatomical Structure,SCT,23451007,Adrenal gland,SCT,24028007,Right
2,aorta,SCT,123037004,Anatomical Structure,SCT,15825003,Aorta,,,
3,autochthon_left,SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,7771000,Left
4,autochthon_right,SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,24028007,Right
...,...,...,...,...,...,...,...,...,...,...
99,vertebrae_T5,SCT,123037004,Anatomical Structure,SCT,56401006,T5 vertebra,,,
100,vertebrae_T6,SCT,123037004,Anatomical Structure,SCT,45296009,T6 vertebra,,,
101,vertebrae_T7,SCT,123037004,Anatomical Structure,SCT,62487009,T7 vertebra,,,
102,vertebrae_T8,SCT,123037004,Anatomical Structure,SCT,11068009,T8 vertebra,,,


###**MhubAI's dcmqi json config as a template**

In [13]:
try:
  os.remove(f'{curr_dir}/dicomseg_metadata_whole.py')
except OSError:
  pass
!wget https://raw.githubusercontent.com/vkt1414/models/main/models/totalsegmentator/config/dicomseg_metadata_whole.json

with open('/content/dicomseg_metadata_whole.json', 'r') as file:
    json_data = json.load(file)

mhubai = pd.json_normalize(json_data['segmentAttributes'][0]).reset_index(drop=True)
mhubai=mhubai[['SegmentAlgorithmName', 'SegmentAlgorithmType', 'SegmentDescription',
       'labelID', 'recommendedDisplayRGBValue',]]
mhubai

--2023-04-07 11:17:10--  https://raw.githubusercontent.com/vkt1414/models/main/models/totalsegmentator/config/dicomseg_metadata_whole.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80391 (79K) [text/plain]
Saving to: ‘dicomseg_metadata_whole.json’

dicomseg_metadata_w 100%[===================>]  78.51K  --.-KB/s    in 0.008s  

2023-04-07 11:17:11 (9.23 MB/s) - ‘dicomseg_metadata_whole.json’ saved [80391/80391]



,SegmentAlgorithmName,SegmentAlgorithmType,SegmentDescription,labelID,recommendedDisplayRGBValue
0,TotalSegmentator,AUTOMATIC,Left adrenal gland,12,"[199, 27, 27]"
1,TotalSegmentator,AUTOMATIC,Right adrenal gland,11,"[199, 27, 27]"
2,TotalSegmentator,AUTOMATIC,Aorta,7,"[224, 97, 76]"
3,TotalSegmentator,AUTOMATIC,Left autochthonous back muscle,100,"[192, 104, 88]"
4,TotalSegmentator,AUTOMATIC,Right autochthonous back muscle,101,"[192, 104, 88]"
...,...,...,...,...,...
99,TotalSegmentator,AUTOMATIC,Vertebrae T5,30,"[241, 214, 145]"
100,TotalSegmentator,AUTOMATIC,Vertebrae T6,29,"[241, 214, 145]"
101,TotalSegmentator,AUTOMATIC,Vertebrae T7,28,"[241, 214, 145]"
102,TotalSegmentator,AUTOMATIC,Vertebrae T8,27,"[241, 214, 145]"


###**Generating Updated dcmqi json config using Slicer Mappings**

In [14]:
merged_df = pd.merge(mhubai,df, left_index=True, right_index=True)
merged_df = merged_df.drop('Structure', axis=1)
merged_df

,SegmentAlgorithmName,SegmentAlgorithmType,SegmentDescription,labelID,recommendedDisplayRGBValue,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning
0,TotalSegmentator,AUTOMATIC,Left adrenal gland,12,"[199, 27, 27]",SCT,123037004,Anatomical Structure,SCT,23451007,Adrenal gland,SCT,7771000,Left
1,TotalSegmentator,AUTOMATIC,Right adrenal gland,11,"[199, 27, 27]",SCT,123037004,Anatomical Structure,SCT,23451007,Adrenal gland,SCT,24028007,Right
2,TotalSegmentator,AUTOMATIC,Aorta,7,"[224, 97, 76]",SCT,123037004,Anatomical Structure,SCT,15825003,Aorta,,,
3,TotalSegmentator,AUTOMATIC,Left autochthonous back muscle,100,"[192, 104, 88]",SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,7771000,Left
4,TotalSegmentator,AUTOMATIC,Right autochthonous back muscle,101,"[192, 104, 88]",SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,24028007,Right
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,TotalSegmentator,AUTOMATIC,Vertebrae T5,30,"[241, 214, 145]",SCT,123037004,Anatomical Structure,SCT,56401006,T5 vertebra,,,
100,TotalSegmentator,AUTOMATIC,Vertebrae T6,29,"[241, 214, 145]",SCT,123037004,Anatomical Structure,SCT,45296009,T6 vertebra,,,
101,TotalSegmentator,AUTOMATIC,Vertebrae T7,28,"[241, 214, 145]",SCT,123037004,Anatomical Structure,SCT,62487009,T7 vertebra,,,
102,TotalSegmentator,AUTOMATIC,Vertebrae T8,27,"[241, 214, 145]",SCT,123037004,Anatomical Structure,SCT,11068009,T8 vertebra,,,


In [22]:
# merged_df_json = json.loads(merged_df.to_json(orient='records'))

# for segment in merged_df_json:
#     segment['SegmentedPropertyCategoryCodeSequence'] = {
#         'CodingSchemeDesignator': segment.pop('SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator'),
#         'CodeValue': segment.pop('SegmentedPropertyCategoryCodeSequence.CodeValue'),
#         'CodeMeaning': segment.pop('SegmentedPropertyCategoryCodeSequence.CodeMeaning')
#     }
#     segment['SegmentedPropertyTypeCodeSequence'] = {
#         'CodingSchemeDesignator': segment.pop('SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator'),
#         'CodeValue': segment.pop('SegmentedPropertyTypeCodeSequence.CodeValue'),
#         'CodeMeaning': segment.pop('SegmentedPropertyTypeCodeSequence.CodeMeaning')
#     }
#     segment['SegmentedPropertyTypeModifierCodeSequence'] = {
#         'CodingSchemeDesignator': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'),
#         'CodeValue': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodeValue'),
#         'CodeMeaning': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodeMeaning')
#     }

merged_df_json = json.loads(merged_df.to_json(orient='records'))

for segment in merged_df_json:
    segment['SegmentedPropertyCategoryCodeSequence'] = {
        'CodingSchemeDesignator': segment.pop('SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator'),
        'CodeValue': segment.pop('SegmentedPropertyCategoryCodeSequence.CodeValue'),
        'CodeMeaning': segment.pop('SegmentedPropertyCategoryCodeSequence.CodeMeaning')
    }
    segment['SegmentedPropertyTypeCodeSequence'] = {
        'CodingSchemeDesignator': segment.pop('SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator'),
        'CodeValue': segment.pop('SegmentedPropertyTypeCodeSequence.CodeValue'),
        'CodeMeaning': segment.pop('SegmentedPropertyTypeCodeSequence.CodeMeaning')
    }
    if ('SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator' in segment and
        (segment['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'].strip() or
         segment['SegmentedPropertyTypeModifierCodeSequence.CodeValue'].strip() or
         segment['SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'].strip())):
        segment['SegmentedPropertyTypeModifierCodeSequence'] = {
            'CodingSchemeDesignator': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator'),
            'CodeValue': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodeValue'),
            'CodeMeaning': segment.pop('SegmentedPropertyTypeModifierCodeSequence.CodeMeaning')
        }
    else:
        for key in ['SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeValue',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeMeaning']:
            if key in segment:
                del segment[key]
final_json = {
    "BodyPartExamined": "WHOLEBODY",
    "ClinicalTrialCoordinatingCenterName": "dcmqi",
    "ClinicalTrialSeriesID": "0",
    "ClinicalTrialTimePointID": "1",
    "ContentCreatorName": "IDC",
    "ContentDescription": "Image segmentation",
    "ContentLabel": "SEGMENTATION",
    "InstanceNumber": "1",
    "SeriesDescription": "Segmentation",
    "SeriesNumber": "42",
    "segmentAttributes": [merged_df_json]
}
with open('dicomseg_metadata_whole_slicerAsRef.json', 'w') as file:
    json.dump(final_json, file)
final_json

{'BodyPartExamined': 'WHOLEBODY',
 'ClinicalTrialCoordinatingCenterName': 'dcmqi',
 'ClinicalTrialSeriesID': '0',
 'ClinicalTrialTimePointID': '1',
 'ContentCreatorName': 'IDC',
 'ContentDescription': 'Image segmentation',
 'ContentLabel': 'SEGMENTATION',
 'InstanceNumber': '1',
 'SeriesDescription': 'Segmentation',
 'SeriesNumber': '42',
 'segmentAttributes': [[{'SegmentAlgorithmName': 'TotalSegmentator',
    'SegmentAlgorithmType': 'AUTOMATIC',
    'SegmentDescription': 'Left adrenal gland',
    'labelID': 12,
    'recommendedDisplayRGBValue': [199, 27, 27],
    'SegmentedPropertyCategoryCodeSequence': {'CodingSchemeDesignator': 'SCT',
     'CodeValue': '123037004',
     'CodeMeaning': 'Anatomical Structure'},
    'SegmentedPropertyTypeCodeSequence': {'CodingSchemeDesignator': 'SCT',
     'CodeValue': '23451007',
     'CodeMeaning': 'Adrenal gland'},
    'SegmentedPropertyTypeModifierCodeSequence': {'CodingSchemeDesignator': 'SCT',
     'CodeValue': '7771000',
     'CodeMeaning': 'Le

In [23]:
import yaml
with open('dicomseg_metadata_whole_slicerAsRef.yaml', 'w') as file:
    yaml.dump(final_json, file)


###**Verifying Slicer Mappings against Snomed CT mappings**

In [ ]:
import requests
import pandas as pd

base_url = 'https://browser.ihtsdotools.org/snowstorm/snomed-ct/MAIN'
headers = { 'User-Agent': 'Python'}

# Replace this with the actual language reference set id for your desired language
language_refset = '900000000000509007'

# Replace these with the actual column names from your dataframe
sct_codes = ['SegmentedPropertyCategoryCodeSequence.CodeValue', 'SegmentedPropertyTypeCodeSequence.CodeValue', 'SegmentedPropertyTypeModifierCodeSequence.CodeValue']

for sct_code_column in sct_codes:
    # Create new columns for preferred and acceptable terms
    preferred_terms_column = sct_code_column + '.PreferredTerms'
    acceptable_terms_column = sct_code_column + '.AcceptableTerms'
    df[preferred_terms_column] = None
    df[acceptable_terms_column] = None
    
    # Get unique SCT codes in column
    unique_sct_codes = df[sct_code_column].unique()
    
    # Get preferred and acceptable terms for each unique SCT code
    sct_code_terms = {}
    for sct_code in unique_sct_codes:
        if pd.isnull(sct_code):
            # If SCT code is missing, set preferred and acceptable terms to empty arrays
            sct_code_terms[sct_code] = ([], [])
        else:
            # Get all descriptions for the given SCT code
            response = requests.get(f'{base_url}/descriptions?conceptId={sct_code}&limit=50', headers=headers)
            descriptions = response.json()['items']
            
            # Filter descriptions by acceptability and activity
            preferred_terms = [desc['term'] for desc in descriptions if desc['active'] and desc['acceptabilityMap'].get(language_refset) == 'PREFERRED']
            acceptable_terms = [desc['term'] for desc in descriptions if desc['active'] and desc['acceptabilityMap'].get(language_refset) == 'ACCEPTABLE']
            
            # Store terms for SCT code
            sct_code_terms[sct_code] = (preferred_terms, acceptable_terms)
    
    # Add arrays to dataframe
    for index, row in df.iterrows():
        sct_code = row[sct_code_column]
        preferred_terms, acceptable_terms = sct_code_terms[sct_code]
        df.at[index, preferred_terms_column] = preferred_terms
        df.at[index, acceptable_terms_column] = acceptable_terms

sct_code_column = 'SegmentedPropertyTypeModifierCodeSequence.CodeValue'
preferred_terms_column = sct_code_column + '.PreferredTerms'
acceptable_terms_column = sct_code_column + '.AcceptableTerms'

# Remove or replace unexpected values in the preferred terms column
for index, row in df.iterrows():
    sct_code = row[sct_code_column]
    if sct_code == '':
        # If SCT code is an empty string, set preferred and acceptable terms to empty arrays
        df.at[index, preferred_terms_column] = []
        df.at[index, acceptable_terms_column] = []

column_order = ['Structure',
                'SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator',
                'SegmentedPropertyCategoryCodeSequence.CodeValue',
                'SegmentedPropertyCategoryCodeSequence.CodeMeaning',
                'SegmentedPropertyCategoryCodeSequence.CodeValue.PreferredTerms',
                'SegmentedPropertyCategoryCodeSequence.CodeValue.AcceptableTerms',
                'SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator',
                'SegmentedPropertyTypeCodeSequence.CodeValue',
                'SegmentedPropertyTypeCodeSequence.CodeMeaning',
                'SegmentedPropertyTypeCodeSequence.CodeValue.PreferredTerms',
                'SegmentedPropertyTypeCodeSequence.CodeValue.AcceptableTerms',
                'SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator',
                'SegmentedPropertyTypeModifierCodeSequence.CodeValue',
                'SegmentedPropertyTypeModifierCodeSequence.CodeMeaning',
                'SegmentedPropertyTypeModifierCodeSequence.CodeValue.PreferredTerms',
                'SegmentedPropertyTypeModifierCodeSequence.CodeValue.AcceptableTerms']

new_column_names = {'SegmentedPropertyCategoryCodeSequence.CodeValue.PreferredTerms': 'SegmentedPropertyCategoryCodeSequence.CodeValue.SNOMEDCTPreferredTerms',
                    'SegmentedPropertyCategoryCodeSequence.CodeValue.AcceptableTerms': 'SegmentedPropertyCategoryCodeSequence.CodeValue.SNOMEDCTAcceptableTerms',
                    'SegmentedPropertyTypeCodeSequence.CodeValue.PreferredTerms': 'SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTPreferredTerms',
                    'SegmentedPropertyTypeCodeSequence.CodeValue.AcceptableTerms': 'SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTAcceptableTerms',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeValue.PreferredTerms': 'SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTPreferredTerms',
                    'SegmentedPropertyTypeModifierCodeSequence.CodeValue.AcceptableTerms': 'SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTAcceptableTerms'}

# Rename the columns of the DataFrame
df = df.rename(columns=new_column_names)
df

,Structure,SegmentedPropertyCategoryCodeSequence.CodingSchemeDesignator,SegmentedPropertyCategoryCodeSequence.CodeValue,SegmentedPropertyCategoryCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning,SegmentedPropertyCategoryCodeSequence.CodeValue.SNOMEDCTPreferredTerms,SegmentedPropertyCategoryCodeSequence.CodeValue.SNOMEDCTAcceptableTerms,SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTPreferredTerms,SegmentedPropertyTypeCodeSequence.CodeValue.SNOMEDCTAcceptableTerms,SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTPreferredTerms,SegmentedPropertyTypeModifierCodeSequence.CodeValue.SNOMEDCTAcceptableTerms
0,adrenal_gland_left,SCT,123037004,Anatomical Structure,SCT,23451007,Adrenal gland,SCT,7771000,Left,"[Body structure (body structure), Body structure]",[Body structures],"[Adrenal structure (body structure), Adrenal s...",[Adrenal gland],"[Left (qualifier value), Left]","[Left lateral, Left side, Lt - Left, Levo-]"
1,adrenal_gland_right,SCT,123037004,Anatomical Structure,SCT,23451007,Adrenal gland,SCT,24028007,Right,"[Body structure (body structure), Body structure]",[Body structures],"[Adrenal structure (body structure), Adrenal s...",[Adrenal gland],"[Right (qualifier value), Right]","[Dextro, Right lateral, Rt - Right, Right side]"
2,aorta,SCT,123037004,Anatomical Structure,SCT,15825003,Aorta,,,,"[Body structure (body structure), Body structure]",[Body structures],"[Aortic structure, Aortic structure (body stru...",[Aorta],[],[]
3,autochthon_left,SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,7771000,Left,"[Body structure (body structure), Body structure]",[Body structures],"[Structure of erector spinae muscle, Structure...",[Erector spinae muscle],"[Left (qualifier value), Left]","[Left lateral, Left side, Lt - Left, Levo-]"
4,autochthon_right,SCT,123037004,Anatomical Structure,SCT,44947003,Erector spinae muscle,SCT,24028007,Right,"[Body structure (body structure), Body structure]",[Body structures],"[Structure of erector spinae muscle, Structure...",[Erector spinae muscle],"[Right (qualifier value), Right]","[Dextro, Right lateral, Rt - Right, Right side]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,vertebrae_T5,SCT,123037004,Anatomical Structure,SCT,56401006,T5 vertebra,,,,"[Body structure (body structure), Body structure]",[Body structures],"[Bone structure of T5 (body structure), Bone s...","[Fifth thoracic vertebra, Fifth dorsal vertebra]",[],[]
100,vertebrae_T6,SCT,123037004,Anatomical Structure,SCT,45296009,T6 vertebra,,,,"[Body structure (body structure), Body structure]",[Body structures],"[Bone structure of T6 (body structure), Bone s...","[Sixth thoracic vertebra, Sixth dorsal vertebra]",[],[]
101,vertebrae_T7,SCT,123037004,Anatomical Structure,SCT,62487009,T7 vertebra,,,,"[Body structure (body structure), Body structure]",[Body structures],"[Bone structure of T7 (body structure), Bone s...","[Seventh thoracic vertebra, Seventh dorsal ver...",[],[]
102,vertebrae_T8,SCT,123037004,Anatomical Structure,SCT,11068009,T8 vertebra,,,,"[Body structure (body structure), Body structure]",[Body structures],"[Bone structure of T8, Bone structure of T8 (b...","[Eighth thoracic vertebra, Eighth dorsal verte...",[],[]


In [ ]:
df.to_csv('slicerMappingsToSnomedMappings.csv')

###**Trying to use Slicer's json config (only has 87 parts)**

In [ ]:
try:
  os.remove(f'{curr_dir}/SegmentationCategoryTypeModifier-TotalSegmentator.term.json')
except OSError:
  pass
!wget https://raw.githubusercontent.com/lassoan/SlicerTotalSegmentator/main/TotalSegmentator/Resources/SegmentationCategoryTypeModifier-TotalSegmentator.term.json




--2023-04-07 08:33:41--  https://raw.githubusercontent.com/lassoan/SlicerTotalSegmentator/main/TotalSegmentator/Resources/SegmentationCategoryTypeModifier-TotalSegmentator.term.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55864 (55K) [text/plain]
Saving to: ‘SegmentationCategoryTypeModifier-TotalSegmentator.term.json’

SegmentationCategor 100%[===================>]  54.55K  --.-KB/s    in 0.007s  

2023-04-07 08:33:41 (7.44 MB/s) - ‘SegmentationCategoryTypeModifier-TotalSegmentator.term.json’ saved [55864/55864]



In [ ]:
import json
import pandas as pd

with open('SegmentationCategoryTypeModifier-TotalSegmentator.term.json', 'r') as file:
    json_data = json.load(file)

rows = []
for category in json_data['SegmentationCodes']['Category']:
    for item in category['Type']:
        row = {}
        row['CodeValue'] = item['CodeValue']
        row['CodeMeaning'] = item['CodeMeaning']
        row['CodingSchemeDesignator'] = item['CodingSchemeDesignator']
        if 'Modifier' in item:
            for modifier in item['Modifier']:
                new_row = row.copy()
                new_row['ModifierCodeValue'] = modifier['CodeValue']
                new_row['ModifierCodeMeaning'] = modifier['CodeMeaning']
                new_row['ModifierCodingSchemeDesignator'] = modifier['CodingSchemeDesignator']
                if '3dSlicerLabel' in modifier:
                    new_row['3dSlicerLabel'] = modifier['3dSlicerLabel']
                if 'recommendedDisplayRGBValue' in modifier:
                    new_row['recommendedDisplayRGBValue'] = modifier['recommendedDisplayRGBValue']
                rows.append(new_row)
        else:
            rows.append(row)

df = pd.DataFrame(rows)

# Rename the columns to match the desired column names
df = df.rename(columns={
    'CodingSchemeDesignator': 'SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator',
    'CodeValue': 'SegmentedPropertyTypeCodeSequence.CodeValue',
    'CodeMeaning': 'SegmentedPropertyTypeCodeSequence.CodeMeaning',
    'ModifierCodingSchemeDesignator': 'SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator',
    'ModifierCodeValue': 'SegmentedPropertyTypeModifierCodeSequence.CodeValue',
    'ModifierCodeMeaning': 'SegmentedPropertyTypeModifierCodeSequence.CodeMeaning'
})
filtered_df = df[pd.notna(df['3dSlicerLabel'])]
filtered_df

,SegmentedPropertyTypeCodeSequence.CodeValue,SegmentedPropertyTypeCodeSequence.CodeMeaning,SegmentedPropertyTypeCodeSequence.CodingSchemeDesignator,SegmentedPropertyTypeModifierCodeSequence.CodeValue,SegmentedPropertyTypeModifierCodeSequence.CodeMeaning,SegmentedPropertyTypeModifierCodeSequence.CodingSchemeDesignator,3dSlicerLabel,recommendedDisplayRGBValue
1,79601000,Scapula,SCT,24028007,Right,SCT,right scapula,"[212, 188, 102]"
2,79601000,Scapula,SCT,7771000,Left,SCT,left scapula,"[212, 188, 102]"
4,85050009,Humerus,SCT,24028007,Right,SCT,right humerus,"[205, 179, 108]"
5,85050009,Humerus,SCT,7771000,Left,SCT,left humerus,"[205, 179, 108]"
7,71341001,Femur,SCT,24028007,Right,SCT,right femur,"[255, 238, 170]"
8,71341001,Femur,SCT,7771000,Left,SCT,left femur,"[255, 238, 170]"
10,44947003,Erector spinae muscle,SCT,24028007,Right,SCT,right erector spinae muscle,"[171, 85, 68]"
11,44947003,Erector spinae muscle,SCT,7771000,Left,SCT,left erector spinae muscle,"[171, 85, 68]"
16,73634005,Common iliac artery,SCT,24028007,Right,SCT,right common iliac artery,"[216, 101, 79]"
17,73634005,Common iliac artery,SCT,7771000,Left,SCT,left common iliac artery,"[216, 101, 79]"
